In [2]:
def Sentimenatalanalyzer(test_data):
    tokens = tokenizer.encode(test_data, return_tensors='pt')
    result = model(tokens)
    result.logits
    final = int(torch.argmax(result.logits))+1
    if final >= 2.5:
        return 1
    else:
        return 0

import speech_recognition as sr 
import random
import playsound
from gtts import gTTS
from time import ctime 
import webbrowser 
import time
import os 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

r = sr.Recognizer()

text=''
voice_data=''
# listen for audio and convert it to text:
def record_audio():
    text=""
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, duration=5)
        audio=r.listen(source)
        try:
            text=r.recognize_google(audio)
        except:
            print('Check internet connectivity')
    print(text)
    return text


def alexis_speak(audio_string):
    tts = gTTS(text=audio_string, lang='en')
    r = random.randint(1, 10000000)
    audio_file = 'audio-' + str(r) + '.mp3'
    tts.save(audio_file)
    playsound.playsound(audio_file)
    print(audio_string)
    os.remove(audio_file)


def respondwithsentiment(voice_data):
    alexis_speak(voice_data)
    with sr.Microphone() as source:
        audio=r.listen(source)
    text=r.recognize_google(audio)
    text=Sentimenatalanalyzer(text.lower())
    return text

def respondwithoutsentiment(voice_data):
    alexis_speak(voice_data)
    with sr.Microphone() as source:
        audio=r.listen(source)
    text=r.recognize_google(audio)
    number_text=text.split()
    max_number = 0
    for text in number_text:    
        try:
            num=int(text)
            if max_number < num:
                max_number=num
        except:  
            pass
    return max_number
            
time.sleep(1)


def conversation():
    matrix=dict()
    question1 = "How old are you?";
    question2 = "Have you traveled to (or living in) any of the COVID-19 affected areas/countries in the last 14 days?";
    question3 = "Did you have any close contact with a person who is known to have COVID-19 during the last 14 days?";
    question4 = "Do you have a sore throat?";
    question5 = "Can you take any smell from food/environment?";
    question6 = "Do you have any pain in muscle?";
    question7 = "Do you feel any illness or discomfort?";
    question8 = "Do you have any mild fever?";
    question9 = "how many days did you have mild fever?"
    question10 = "what is your current body temperature?"
    question11 = "Do you have any dry cough?";
    question12 = "how many days did you have dry cough?";
    question13 = "Do you have any headache?";
    question14 = "how many days did you have headache?";
    question15 = "Do you have Diarrhea?";
    question16 = "how many days did you have Diarrhea?";
    question17 = "Do you have any tachypnoea and shortness of breath?";
    question18 = "How many days did you have shortness of breath?";
    matrix["age"]=respondwithoutsentiment(question1)
    matrix["travel"]=respondwithsentiment(question2)
    matrix["closecontact"]=respondwithsentiment(question3)
    matrix["sorethroat"]=respondwithsentiment(question4)
    matrix["foodsmell"]=respondwithsentiment(question5) 
    matrix["musclepain"]=respondwithsentiment(question6)
    matrix["discomfort"]=respondwithsentiment(question7)
    matrix["fever"]=respondwithsentiment(question8)
    if matrix["fever"] == 1:
        matrix["feverdays"]=respondwithoutsentiment(question9)
        while matrix["feverdays"] > 15:
            alexis_speak('Please tell the value between 1 to 15')
            matrix["feverdays"]=respondwithoutsentiment(question9)
        matrix["temperature"]=respondwithoutsentiment(question10)
        while matrix["temperature"] <= 105 and matrix["temperature"] >= 97:
            alexis_speak('Please tell the value between 97 to 105')
            matrix["temperature"]=respondwithoutsentiment(question10)
    else:
        matrix["feverdays"] = 0
        matrix["temperature"] = 0
    matrix["drycough"]=respondwithsentiment(question11)
    if matrix["drycough"] == 1:
        matrix["drycoughdays"]=respondwithoutsentiment(question12)
        while matrix["drycoughdays"] > 15:
            alexis_speak('Please tell the value between 1 to 15')
            matrix["drycoughdays"]=respondwithoutsentiment(question12)
    else:
        matrix["drycoughdays"] = 0
    matrix["headache"]=respondwithsentiment(question13)
    if matrix["headache"] == 1:
        matrix["headachedays"]=respondwithoutsentiment(question14)
        while matrix["headachedays"] > 15:
            alexis_speak('Please tell the value between 1 to 15')
            matrix["headachedays"]=respondwithoutsentiment(question14)
    else:
        matrix["headachedays"] = 0
    matrix["diarrhea"]=respondwithsentiment(question15)
    if matrix["diarrhea"] == 1:
        matrix["diarrheadays"]=respondwithoutsentiment(question16)
        while matrix["diarrheadays"] > 15:
            alexis_speak('Please tell the value between 1 to 15')
            matrix["diarrheadays"]=respondwithoutsentiment(question16)
    else:
        matrix["diarrheadays"] = 0
    matrix["breath"]=respondwithsentiment(question17)
    if matrix["breath"] == 1:
        matrix["breathdays"]=respondwithoutsentiment(question18)
        while matrix["breathdays"] > 15:
            alexis_speak('Please tell the value between 1 to 15')
            matrix["breathdays"]=respondwithoutsentiment(question18)
    else:
        matrix["breathdays"] = 0
    return matrix

def csv(matrix):
    import csv
    my_dict = matrix
    feature_cols = ['age','travel', 'closecontact', 'sorethroat', 'foodsmell',
       'musclepain', 'discomfort', 'fever','feverdays','temperature', 
        'drycough','drycoughdays', 'headache', 'headachedays', 'diarrhea',
       'diarrheadays', 'breath','breathdays']
    with open('test.csv', 'w') as f:
        for cols in feature_cols:
            if cols in my_dict.keys():
                f.write("%s,"%(cols))
        f.write("\n")
        for cols in feature_cols:
            if cols in my_dict.keys():
                f.write("%s,"%(my_dict[cols]))

def prediction():
    import pickle
    import pandas as pd
    filename = 'finalized_model.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    col_names = ['age','travel', 'closecontact', 'sorethroat', 'foodsmell',
       'musclepain', 'discomfort', 'fever','feverdays','temperature', 
        'drycough','drycoughdays', 'headache', 'headachedays', 'diarrhea',
       'diarrheadays', 'breath','breathdays','infected']
    feature_cols = ['age','travel', 'closecontact', 'sorethroat', 'foodsmell',
       'musclepain', 'discomfort', 'fever','feverdays','temperature', 
        'drycough','drycoughdays', 'headache', 'headachedays', 'diarrhea',
       'diarrheadays', 'breath','breathdays']
    # load dataset
    pima = pd.read_csv('test.csv', header=None, names=col_names)
    pima = pima.apply(pd.to_numeric, errors='coerce')

    # Delete the rows with labels 0,1,5
    pima = pima.drop([0], axis=0)

    #pima.head()
    a = pima[feature_cols] # Features
    #b = pima.infectedornot # Target variable

    #Predict the response for test dataset
    b_pred = loaded_model.predict(a)

    # Model Accuracy, how often is the classifier correct?
    #print("Accuracy:",metrics.accuracy_score(b, b_pred))

    return b_pred[0]

def check_covid(covid_percentage):
    if covid_percentage < 0.5:
        alexis_speak('Your symptoms suggest that the probability of being infected is very low')
    else:
        alexis_speak('Your symptoms suggest that the probability of being infected is very high')

In [6]:
matrix_con=conversation()
csv(matrix_con)
final_result=prediction()
check_covid(final_result)

How old are you?
Have you traveled to (or living in) any of the COVID-19 affected areas/countries in the last 14 days?
Did you have any close contact with a person who is known to have COVID-19 during the last 14 days?
Do you have a sore throat?
Can you take any smell from food/environment?
Do you have any pain in muscle?
Do you feel any illness or discomfort?
Do you have any mild fever?
Do you have any dry cough?
Do you have any headache?
Do you have Diarrhea?
how many days did you have Diarrhea?
Do you have any tachypnoea and shortness of breath?


c:\users\niloy\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
c:\users\niloy\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
c:\users\niloy\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassif

Your symptoms suggest that the probability of being infected is very low


In [7]:
print(matrix_con)

{'age': 20, 'travel': 1, 'closecontact': 0, 'sorethroat': 1, 'foodsmell': 0, 'musclepain': 0, 'discomfort': 0, 'fever': 0, 'feverdays': 0, 'temperature': 0, 'drycough': 0, 'drycoughdays': 0, 'headache': 0, 'headachedays': 0, 'diarrhea': 1, 'diarrheadays': 10, 'breath': 0, 'breathdays': 0}


In [12]:
matrix_con={'age': 23, 'travel': 0, 'closecontact': 1, 'sorethroat': 0, 'foodsmell': 1, 'musclepain': 0, 'discomfort': 1, 'fever': 1, 'feverdays': '5', 'temperature': '100', 'drycough': 0, 'drycoughdays': 0, 'headache': 1, 'headachedays': '10', 'diarrhea': 0, 'diarrheadays': 0, 'breath': 1, 'breathdays': '9'}

In [14]:
csv(matrix_con)

In [15]:
final_result=prediction()
check_covid(final_result)

Your symptoms suggest that the probability of being infected is very high


In [35]:
r = sr.Recognizer()
with sr.Microphone() as source:
    r.adjust_for_ambient_noise(source, duration=5)
    audio=r.listen(source)
text=r.recognize_google(audio)
number_text=text.split()
max_number = 0
for text in number_text:    
    try:
        num=int(text)
        if max_number < num:
            max_number=num
    except:  
        print('')